In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import sweetviz as vs

In [6]:
%store df

UsageError: Unknown variable 'df'


In [ ]:
df_compl = pd.read_csv("Lending Club/loans_2007.csv")

In [ ]:
df_compl.isnull().sum()

In [ ]:
df_compl.info()

In [ ]:
# wir elimierniern drei übeltäter die voll mit nans sind in der zeile
rows_delete=df_compl[df_compl["loan_amnt"].isnull()].index


df_compl= df_compl.drop(rows_delete, axis=0)

In [ ]:
####################################### Data cleaning
# emp_legth,home_ownership verdummien
# interaktionterm von annual income und verifications_status
# untersuchen ob zwischen 2007 und 2009 kreditausfälle stattgefunden haben durch die krise, varibale issue_d untersuchen
# loan_status zu einer neuen kategorievariable machen mit : 0 = gezahlt, 1= ausstehend, 2= nicht in der lage
# die, die wir rausgeschmissen haben, haben wir entweder nicht verstanden oder waren unbalancierte variablen 
# emp_length verdummien, da ca. 1100 fehlen


columns_delete=["id", "member_id", "grade", "emp_title","pymnt_plan","title","zip_code", "addr_state","delinq_2yrs","earliest_cr_line","inq_last_6mths","pub_rec","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","last_pymnt_amnt","last_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","policy_code","application_type","acc_now_delinq","chargeoff_within_12_mths","delinq_amnt","tax_liens"]

y_columns=["loan_status","sub_grade"]




In [ ]:
# hier wird das y, also unser target bestimmt
y_columns=["loan_status","sub_grade"]
y_target=df_compl[y_columns]

y_app= y_target["loan_status"].apply(lambda y: 1 if y == y_01[0] else 0)





In [ ]:
df_compl.isnull().sum()

In [ ]:
# weitere features werden gelöscht aufgrund correlations
columns_corr=["funded_amnt","funded_amnt_inv","total_pymnt_inv"]


In [ ]:
# weitere features werden gelöscht, da sie die selben infos tragen bzw. daraus werden unsere anderen variablen bestimmt
columns_sameinfo=["total_rec_int","total_rec_prncp","total_rec_late_fee", "recoveries","collection_recovery_fee"]

In [ ]:
#spalten löschen
df= df_compl.drop(columns_delete, axis=1).drop(y_colmuns, axis=1).drop(columns_corr, axis=1).drop(columns_sameinfo, axis=1)

In [ ]:
#### Hier werden die dateintypen, welche falsch sind korrigiert
if type(df["int_rate"][0])== type("str"): 
    df["int_rate"]=df["int_rate"].str.strip("%").astype("float")
else:
    print("Already a float")

    

    
if type(df["term"][0]) == type("str"): 
    df["term"]=df["term"].str.strip("months").astype("float")
else:
    print("Already a float")


In [ ]:
#### hier wird eine dateityp zum zeitdatentyp geändert
df["issue_d"]= pd.to_datetime(df["issue_d"])

In [ ]:
df["issue_d"]

In [ ]:
import datetime

df['issue_d'] = pd.to_datetime(df["issue_d"], format="%Y-%M-%D")
df['issue_d'] = df['issue_d'].map(datetime.datetime.toordinal)


In [ ]:
# hier werden die fehlenden daten durch wahrscheinliche daten ersetzt (mean,median,blaa)

df["open_acc"].fillna(df["open_acc"].median(),inplace=True)
df["annual_inc"].fillna(df["annual_inc"].median().round(2),inplace=True)
df["emp_length"].fillna("no info at all", inplace=True)
df["pub_rec_bankruptcies"].fillna(0.0, inplace=True)



In [ ]:
X_train.describe()

In [ ]:
df.corrwith(df["open_acc"])

In [ ]:
print(df.corr().shape)
fig= plt.figure(figsize=(20,20))
sns.heatmap(df.corr())

In [ ]:
### hier werden dummies erstellt

df_dummie=pd.get_dummies(df, drop_first=True)


df_y_sub_grade= pd.get_dummies(y_target["sub_grade"], drop_first=True)

In [ ]:
y

In [ ]:
######### Modell
yy=y_target["loan_status"]
X= df_dummie

y_01= ["Fully Paid", "Charged Off"]
#y=y.apply(lambda y: 1 if y == y_01[1] else 0)
y= yy.apply(lambda y: 1 if y == y_01[0] else 0)

In [ ]:
%store X y

In [ ]:
y.value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report


In [ ]:
###### Naiv-estimator


# TODO: Total number of records
n_records = len(X)

# TODO: Number of records where individual's income is more than $50,000
fully_paid = y.sum()

# TODO: Number of records where individual's income is at most $50,000
charged_off = n_records - fully_paid

# TODO: Percentage of individuals whose income is more than $50,000
greater_percent = 100 * fully_paid / n_records

# Print the results
print ("Total number of records: {}".format(n_records))
print ("Individuals has paid her credit fully: {}".format(fully_paid))
print ("Individuals hasn't paid her credit: {}".format(charged_off))
print ("Percentage of individuals that paid ther credit: {:.2f}%".format(greater_percent))

In [ ]:
y.sum()

In [ ]:
#Split into train and test

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.4, random_state = 70, stratify= y)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

In [ ]:
#### skalieren
from sklearn.preprocessing import MinMaxScaler


# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc',"issue_d","dti","open_acc","revol_bal","pub_rec_bankruptcies" ]
X_train[numerical] = scaler.fit_transform(X_train[numerical])
X_test[numerical] = scaler.transform(X_test[numerical])

# Show an example of a record with scaling applied
X_train[numerical].sample(1)

In [ ]:
X_train

In [ ]:
####fit von log reg

lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
lr.score(X_train,y_train)

In [ ]:
#print(lr.coef_);
#print(lr.intercept_);

In [ ]:
### predic

y_pred = lr.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,y_pred)
print(cm)
cp= classification_report(y_test,y_pred)
print(cp)
#acs=accuracy_score((y_test,y_pred))
#print(acs)

In [ ]:
######## DecisionTree
dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(X_train,y_train)


In [ ]:
y_pred_dtc=dtc.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,y_pred_dtc)
print(cm)
cp= classification_report(y_test,y_pred_dtc)
print(cp)